**NIGHTTIME LIGHTS DIFFERENCE OF PHILIPPINES DURING 2019, 2020, and 2021**

In [1]:
import geemap, ee
import geemap.eefolium as geemap
import geemap.foliumap as geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()


In [2]:
def viirs(x,y):
    viirs_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate(x, y).select('avg_rad').first()
    return viirs_12

In [3]:
viirs_2019_12 = viirs("2019-12-01", "2019-12-31")
viirs_2020_12 = viirs("2020-12-01", "2020-12-31")
viirs_2021_12 = viirs("2021-12-01", "2021-12-31")


#Getting the geometry of Philippines
phl = ee.Feature(ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Philippines')).first()).geometry()

# clipping the VIIRS image to Philippines
ntl_phl_2019 = viirs_2019_12.clip(phl)
ntl_phl_2020 = viirs_2020_12.clip(phl)
ntl_phl_2021 = viirs_2021_12.clip(phl)


#initializing the map and centering it on the Philippines
Map0 = geemap.Map()
Map0.centerObject(phl, zoom = 6)
Map0.add_basemap('SATELLITE')
Map0.addLayer(ntl_phl_2019, {}, "VIIRS-DNB December 2019")
Map0.addLayer(ntl_phl_2020, {}, "VIIRS-DNB December 2020")
Map0.addLayer(ntl_phl_2021, {}, "VIIRS-DNB December 2021")
Map0.addLayerControl()
Map0

In [4]:
Map1 = geemap.Map()
Map1.centerObject(phl, zoom = 6)
Map1.add_basemap('SATELLITE')
Map1.addLayer(ntl_phl_2019, {}, "VIIRS-DNB December 2019")
Map1.addLayerControl()
Map1

In [5]:
Map2 = geemap.Map()
Map2.centerObject(phl, zoom = 6)
Map2.add_basemap('SATELLITE')
Map2.addLayer(ntl_phl_2020, {}, "VIIRS-DNB December 2020")
Map2.addLayerControl()
Map2

In [6]:
Map3 = geemap.Map()
Map3.centerObject(phl, zoom = 6)
Map3.add_basemap('SATELLITE')
Map3.addLayer(ntl_phl_2021, {}, "VIIRS-DNB December 2021")
Map3.addLayerControl()
Map3

Using the reduceRegion function to get the mean and standard deviation of a region

In [7]:
ntl_phl_2019 = viirs_2019_12.clip(phl)
ntl_phl_2020 = viirs_2020_12.clip(phl)
ntl_phl_2021 = viirs_2021_12.clip(phl)

def mean(x):
    mean = x.reduceRegion(reducer = ee.Reducer.mean())
    return mean


def std(x):
    std = x.reduceRegion(reducer= ee.Reducer.stdDev())
    return std

mean_2019 = mean(ntl_phl_2019)
mean_2020 = mean(ntl_phl_2020)
mean_2021 = mean(ntl_phl_2021)
std_2019 = std(ntl_phl_2019)
std_2020 = std(ntl_phl_2020)
std_2021 = std(ntl_phl_2021)


In [8]:
def num_mean(x):
    num_mean = ee.Number(x.get('avg_rad'))
    return num_mean 

def num_stdev(x):
    num_stdev = ee.Number(x.get('avg_rad'))
    return num_stdev 

num_mean_2019 = num_mean(mean_2019)
num_mean_2020 = num_mean(mean_2020)
num_mean_2021 = num_mean(mean_2021)
num_std_2019 = num_stdev(std_2019)
num_std_2020 = num_stdev(std_2020)
num_std_2021 = num_stdev(std_2021)

In [9]:
print(f" the mean avg radiance (per month) for the pixel in the Philippines during 2019 is : {num_mean_2019.getInfo():.4f}")
print(f" the mean avg radiance (per month) for the pixel in the Philippines during 2020 is : {num_mean_2020.getInfo():.4f}")
print(f" the mean avg radiance (per month) for the pixel in the Philippines during 2021 is : {num_mean_2021.getInfo():.4f}")

 the mean avg radiance (per month) for the pixel in the Philippines during 2019 is : 0.3668
 the mean avg radiance (per month) for the pixel in the Philippines during 2020 is : 0.4550
 the mean avg radiance (per month) for the pixel in the Philippines during 2021 is : 0.4887


In [10]:
print(f"the std dev for 2019 is: {num_std_2019.getInfo():.4f}")
print(f"the std dev for 2020 is: {num_std_2020.getInfo():.4f}")
print(f"the std dev for 2021 is: {num_std_2021.getInfo():.4f}")

the std dev for 2019 is: 2.5709
the std dev for 2020 is: 1.7730
the std dev for 2021 is: 2.0321


From the first 4 maps above, we can make a cleaner layer by standardizing the image. This is done by subtracting the mean of the entire image from each pixel's value and the dividing it to the standard deviation of the entire image. 

In [11]:
# ntl_phl_2019_std = ntl_phl_2019.subtract(num_mean_2019).divide(num_std_2019)

#creating function

def standardize(x,mean,std):
    standardize = x.subtract(mean).divide(std)
    return standardize

ntl_phl_2019_norm = standardize(ntl_phl_2019, num_mean_2019, num_std_2019)
ntl_phl_2020_norm = standardize(ntl_phl_2020, num_mean_2020, num_std_2020)
ntl_phl_2021_norm = standardize(ntl_phl_2021, num_mean_2021, num_std_2021)


In [15]:
#creating split panels for year 2019
left_2019 = geemap.ee_tile_layer(ntl_phl_2019, {}, "VIIRS-DNB December 2019" ) 
right_2019 = geemap.ee_tile_layer(ntl_phl_2019_norm, {}, "VIIRS-DNB December 2019 Normalized")

#creating split panels for year 2020
left_2020 = geemap.ee_tile_layer(ntl_phl_2020, {}, "VIIRS-DNB December 2020" ) 
right_2020 = geemap.ee_tile_layer(ntl_phl_2020_norm, {}, "VIIRS-DNB December 2020 Normalized")

#creating split panels for year 2021
left_2021 = geemap.ee_tile_layer(ntl_phl_2021, {}, "VIIRS-DNB December 2021" ) 
right_2021 = geemap.ee_tile_layer(ntl_phl_2021_norm, {}, "VIIRS-DNB December 2021 Normalized")

#creating functions

def Mapping(left, right):
    Map = geemap.Map()
    Map.centerObject(phl, zoom = 6)
    Map.add_basemap('SATELLITE')
    Map.split_map(left_layer = left, right_layer = right)
    Map.addLayerControl()
    return Map

#Mapping 2019
Map5 = Mapping(left_2019, right_2019)
Map5

In [16]:
#Mapping 2020
Map6 = Mapping(left_2020, right_2020)
Map6

In [17]:
#Mapping 2021
Map7 = Mapping(left_2021, right_2021)
Map7